In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier

from ipywidgets import interactive

from collections import defaultdict


import folium
import re


cols = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
        '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', 
        '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
        '#000075', '#808080']*10

## 1. EXPLORATORY DATA ANALYSIS

In [ ]:
pip install hdbscan

In [ ]:
df = pd.read_csv("../input/taxi-data/taxi_data.csv")
df.head()

<font color="purple">
First we will find out the dublucates and mising values.

In [ ]:
print(f" We have {df.duplicated(subset=['LON','LAT']).sum()} dublicates in the dataset")

In [ ]:
print(f" We have {df.isnull().sum()} missing values in the dataset")

In [ ]:
df.isnull().sum()

In [ ]:
print(f"Dataset shape before dublicates and misisgn values {df.shape}")
df.dropna(inplace=True)
df.drop_duplicates(subset=['LON','LAT'],keep="first",inplace=True)
print(f"Dataset shape after dublicates and misisgn values {df.shape}")

<font color="purple">
As we can see the output above, 5 rows has been dropped from the dataset which is not sp much to affect out future work.

In [ ]:
X = df[['LON','LAT']]
X

## 2.VISUALIZING GEOGRAPHICAL DATA

In [ ]:
sns.jointplot(data=df,x='LON',y='LAT')

In [ ]:
#This looks like California all right, but other than that it is hard to see any particular pattern. 
#Setting the alpha option to 0.1 makes it much easier to visualize the places where there is a high density of data points
df.plot(x="LON",y="LAT",kind="scatter",alpha=0.4,figsize=(20,15))
#Now that’s much better: you can clearly see the high-density areas, namely the Bay Area and around Los Angeles and San Diego

In [ ]:
taxi_map = folium.Map(location=[df["LAT"].mean(),df["LON"].mean(),],zoom_start=9,tiles="Stamen Toner")
taxi_map # This is the map we will use

<font color="purple">
Now we will fill map with the data we have

In [ ]:
taxi_map = folium.Map(location=[df["LAT"].mean(),df["LON"].mean(),],zoom_start=9,tiles="OpenStreetMap")
for row_number, row in df.iterrows():
    folium.CircleMarker(
        location = [row["LAT"],row["LON"]],
        radius = 5,
        popup = df["NAME"],
        color= "#1787FE",
        fill=True,
        fill_color="yellow").add_to(taxi_map)
taxi_map    

## 3. KMEANS CLUSTERING

## 3.1. How The Algorithm Works:

<font color="purple">
The algorithm

  -Choose a number of Clusters K
    
  -Randomly assign each point to a specific cluster
    
  -Until clusters stop changing, repeat the following steps:
    
  -For each cluster, compute the cluster's centroid by taking the mean vector points in the cluster
    
  -Assign each data point to the cluster for which the centroid is the closest

In [ ]:
from IPython.display import Image
url="https://i.stack.imgur.com/FQhxk.jpg"
Image(url,width=800, height=800)

<font color="purple">
The first observation about data is shown in top-left of the figure above.

In the step 1 in the algorithm, each observation is randomly assigned to a cluster.

In the step 2a in the algorithm,the cluster centroid for each cluster is computed, which are shown as large colored disk as shown top-right of the figure.

Initially these centroids are almost overlapping as we can see from the figure because initial cluster assignments are chosen randomly.

In the step 2a in the algorithm(bottom-left of the figure above), each observation is assigned to the neares centroid.

In bottom-center of the figure above, step 2a once again is performed which lead to new cluster centroids.

We basically keep repeating these steps until there is no new cluster which means data points are being reassigned to a new cluster centroid.

At the bottom-right, we have the results obtained after about 10 iterations

## 3.2. How to Find Best k cluster:

<font color="purple">
To perform K Mean Clustering, we have to decide how many clusters we expect in the data.

There is no easy answer for choosing a best K Value, but we can use elbow method to achieve a good K Value for the algorithm.

The basic idea behind this method is that it plots the various values of cost with changing k. As the value of K increases, there will be fewer elements in the cluster. So average distortion will decrease. The lesser number of elements means closer to the centroid. So, the point where this distortion declines the most is the elbow point.

First of all, we compute the sum of squared errors(SSE) for some values of K(example, 2,3,4,6,etc).The SSE is the sum of squared distance between each member of the cluster and its centroid.

If we plot K value against the SSE,the error decreases as the K Value increases because if the number of cluster increases, they should be smaller. Accordingly the distortion becomes also smaller.

From this perspective, the idea of using elbow method is to choose K Value at which SSE decreases abruptly, and this produces an elbow effect as we can see in the following picture.

In [ ]:
url="https://i.stack.imgur.com/vc01j.png"
Image(url,width=800, height=800)


<font color="purple">
In the plot above, we see the number of cluster on the x-axis and within group sum of squares.

We try to choose a K Value where we won't get much information by increasing the number of classes, which means that we will not significantly within groups sum of squares by increase the number of clusters.

The number 2 or 3 is the most ideal in this plot because it is the first point the elbow shape happens and avoid to increase the number clusters more.

## 3.3. Implementing K Means Clustering and Evaluation the Performance of the Algorithm

In [ ]:
from sklearn.cluster import KMeans
loss=list()
for i in range(1,100):
    kmeans=KMeans(n_clusters= i, init="k-means++")
    kmeans.fit(X)
    loss.append(kmeans.inertia_)
sns.set_style("darkgrid")
plt.figure(figsize=(12,10))
plt.plot(range(1,100), loss)
plt.title("Elbow Method")
plt.xlabel("Number of Cluster")  
plt.ylabel("loss")
plt.show()

<font color="purple">
As we can see, we can have best cluster value when number of cluster is equal to 16

In [ ]:
kmeans=KMeans(n_clusters=20,init="k-means++")
predicted_clusters=kmeans.fit_predict(X)
silhouette_score(X,predicted_clusters)

In [ ]:
cluster_df=pd.DataFrame(predicted_clusters,columns=["KMeans Clusters"])
new_df=pd.concat([X, cluster_df], axis=1)
new_df

In [ ]:
new_df.dropna(inplace=True)

In [ ]:
new_df.groupby("KMeans Clusters").mean()
#It is apparnt that Annual Income and Spending Score plays important role in the number of clusters

In [ ]:
plt.figure(figsize=(12,10))
sns.scatterplot(x=new_df["LAT"],y= new_df["LON"],hue=new_df["KMeans Clusters"],palette="magma")

In [ ]:
k=20
def create_map(df, cluster_column):
    taxi_map = folium.Map(location=[df["LAT"].mean(), df["LON"].mean()], zoom_start=9, tiles='OpenStreetMap')

    for row_number, row in df.iterrows():

    # get a colour
        #cluster_colour = cols[row[cluster_column]]

        folium.CircleMarker(
                            location= [row["LAT"],row["LON"]],
                            radius=5,
                            popup= row[cluster_column],
                            color="yellow",
                            fill=True,
                            fill_color="blue"
    ).add_to(taxi_map)
    return taxi_map

clustered_map = create_map(new_df, "KMeans Clusters")
print(f'K={k}')
print(f'Silhouette Score: {silhouette_score(X, predicted_clusters)}')

taxi_map.save('kmeans_20.html')


In [ ]:
clustered_map

<font color="purple">
Another method to find best k value is to use silhouette_score. Silhouette score is used to evaluate the quality of clusters created using clustering algorithms such as K-Means in terms of how well samples are clustered with other samples that are similar to each other.

The value of Silhouette score varies from -1 to 1. If the score is 1, the cluster is dense and well-separated than other clusters. A value near 0 represents overlapping clusters with samples very close to the decision boundary of the neighbouring clusters. A negative score [-1, 0] indicate that the samples might have got assigned to the wrong clusters.

In [ ]:
best_silhouette, best_k = -1, 0

for k in tqdm(range(2, 100)):
    model = KMeans(n_clusters=k, random_state=1).fit(X)
    class_predictions = model.predict(X)
    
    curr_silhouette = silhouette_score(X, class_predictions)
    if curr_silhouette > best_silhouette:
        best_k = k
        best_silhouette = curr_silhouette
        
print(f'K={best_k}')
print(f'Silhouette Score: {best_silhouette}') 

## 4. DBSCAN

<font color="purple">
Based on a set of points (let’s think in a bidimensional space as exemplified in the figure), DBSCAN groups together points that are close to each other based on a distance measurement (usually Euclidean distance) and a minimum number of points. It also marks as outliers the points that are in low-density regions.
Parameters:
The DBSCAN algorithm basically requires 2 parameters:
eps: specifies how close points should be to each other to be considered a part of a cluster. It means that if the distance between two points is lower or equal to this value (eps), these points are considered neighbors.
minPoints: the minimum number of points to form a dense region. For example, if we set the minPoints parameter as 5, then we need at least 5 points to form a dense region.

eps: if the eps value chosen is too small, a large part of the data will not be clustered. It will be considered outliers because don’t satisfy the number of points to create a dense region. On the other hand, if the value that was chosen is too high, clusters will merge and the majority of objects will be in the same cluster. The eps should be chosen based on the distance of the dataset (we can use a k-distance graph to find it), but in general small eps values are preferable.
minPoints: As a general rule, a minimum minPoints can be derived from a number of dimensions (D) in the data set, as minPoints ≥ D + 1. Larger values are usually better for data sets with noise and will form more significant clusters. The minimum value for the minPoints must be 3, but the larger the data set, the larger the minPoints value that should be chosen

In [ ]:
len(X)

In [ ]:
model = DBSCAN(eps=0.01, min_samples=5).fit(X)
class_predictions = model.labels_
class_predictions

In [ ]:
kmeans=KMeans(n_clusters=20,init="k-means++")
predicted_clusters=kmeans.fit_predict(X)
silhouette_score(X,predicted_clusters)

## 5.Hierarchical Clustering

<font color="purple">

In [ ]:
from scipy.cluster import hierarchy 
hier=hierarchy.dendrogram(hierarchy.linkage(X, method="ward"))
plt.title("Dendogram of Hierarchical Clustering")
plt.xlabel("Observation Points")
plt.ylabel("Euclidean Distance")
plt.show() 

In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac=AgglomerativeClustering(n_clusters=20, affinity="euclidean",linkage="ward")
agglomerative_clusters= ac.fit_predict(X)
agglomerative_clusters

In [ ]:
silhouette_score(X,agglomerative_clusters)

In [ ]:
df3= pd.DataFrame(agglomerative_clusters, columns=["Agglomerative Clusters"])
new_df=pd.concat([new_df,df3],axis=1)
new_df

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 10), sharey=True)
fig.suptitle('Distribution of Cluster')

# KMeans Clustering
sns.scatterplot(ax=axes[0], x=new_df["LAT"], y=new_df["LON"],hue=new_df["KMeans Clusters"],palette="viridis")
axes[0].set_title("According to KMeans Clusters")
#Agglomerative Clustering
sns.scatterplot(ax=axes[1], x=new_df["LAT"], y=new_df["LON"],hue=new_df["Agglomerative Clusters"],palette="viridis")
axes[0].set_title("According to Agglomerative Clusters")